# Zillow9 - GTO 

## Plots for treated regions 
________

## Threshold x median prices

Counties thresholds were not uniform before Aug, 2018 and their values relative to median prices varied considerably. We investigate house prices around treatment on the following change in policy:
- inclusion of wire transfers 
- change of threshold to US$ 300,000.

The plot bellow illustrates how close/far the threshold was from median prices or median top tier prices for 4th wave (introduced wire transfers) and 6th wave (300.000 threshold).

In [32]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import json
#from linearmodels.panel import PanelOLS
#import statsmodels.api as sm
#from linearmodels.panel import compare
import warnings
warnings.filterwarnings('ignore')

# get color settings
%run functions/nice_plot_pandas_getdata.ipynb

# add save image function
%run functions/save_image_plotly.ipynb

pd.options.display.float_format = '{:.2f}'.format

Function save_plotly(figure, filename)



### Load GTO data for time series
Only 4th ans 6th waves -  all coounties

In [33]:
# load GTO info and set column "date" YYYY-MM (will use as merge key)
GTO = pd.read_csv('.\output\GTO_table_long.csv',
                  dtype = {'FIPS': object,  'threshold':'Int64'},
                  usecols = ['FIPS', 'date enforcement', 'wave', 'threshold'])

GTO = GTO[GTO.wave.isin(['wave_4','wave_6']) & GTO.threshold.notna()]
GTO['Date'] = GTO.loc[:,'date enforcement'].str.slice(stop=7)# YYYY-MM date 
GTO.drop('date enforcement',1).head(3)

,FIPS,wave,threshold,Date
66,36061,wave_4,3000000,2017-08
67,12086,wave_4,1000000,2017-08
68,48029,wave_4,500000,2017-08


### Load GTO Data for map
Only first time treated - will be added to map at each wave

In [34]:
GTO_all = pd.read_csv('.\output\GTO_table_long.csv',
                  dtype = {'FIPS': object,  'threshold':'Int64'},
                  usecols = ['FIPS', 'wave','threshold'])

# take only dates were counties were treated
GTO_all = GTO_all[GTO_all.threshold.notna()].reset_index().drop('threshold',1)

In [35]:
# take only first occurrence
GTO_all = GTO_all.groupby('FIPS').first().reset_index().sort_values('index')

### Load lat/lon positions

In [36]:
# load lat/lon positions
latlon = pd.read_csv('../input2/UScounty_boundaries.csv', dtype = {'geo_id': object},
                    usecols = ['geo_id', 'int_point_lat', 'int_point_lon'])
latlon.columns = ['FIPS', 'lat', 'lon']

### Load prices data (Zillow)

In [146]:
# load price data and format Date as YYYY-MM
def load_Zillow_YYYYMM_noCounty():

    df = (pd.read_csv('../output/tbl_reg.csv',
                      dtype={'FIPS': object,'price_mid':'Int64',  'price_TOP':'Int64'},
                      usecols=['FIPS', 'Date', 'RegionName', 'State',
                               'price_mid', 'price_TOP',
                               'logprice_mid', 'logprice_TOP',
                               'price_pct_mid', 'price_pct_TOP',
                               ]))

    df.Date = df.Date.str.slice(stop=7) # YYYY-MM date 
    df.RegionName = df.RegionName.str.slice(stop=-7) # drop word "County"
    return df
c

### merge GTO + lat/lon positions  + names

In [38]:
# merge positions with GTO data
GTO_waves = GTO_all.merge(latlon, how= 'left')
GTO_waves.head(15)

,FIPS,index,wave,lat,lon
0,36061,0,wave_1,40.78,-73.97
1,12086,1,wave_1,25.61,-80.50
2,48029,24,wave_2,29.45,-98.52
3,12011,25,wave_2,26.19,-80.48
4,12099,26,wave_2,26.65,-80.45
5,36005,27,wave_2,40.85,-73.85
6,36047,28,wave_2,40.64,-73.95
7,36081,29,wave_2,40.66,-73.84
8,36085,30,wave_2,40.56,-74.14
9,06073,31,wave_2,33.02,-116.78


In [39]:
# merge names for hover/annotations
GTO_waves = GTO_waves.merge(df[df.Date== max(df.Date)][['FIPS', 'RegionName', 'State']], how= 'left')
GTO_waves.head(3)

,FIPS,index,wave,lat,lon,RegionName,State
0,36061,0,wave_1,40.78,-73.97,New York,NY
1,12086,1,wave_1,25.61,-80.50,Miami-Dade,FL
2,48029,24,wave_2,29.45,-98.52,Bexar,TX


In [40]:
GTO_waves['county'] = GTO_waves.RegionName + ', ' + GTO_waves.State

In [41]:
df = GTO_waves.copy()

In [42]:
df.head(4)

,FIPS,index,wave,lat,lon,RegionName,State,county
0,36061,0,wave_1,40.78,-73.97,New York,NY,"New York, NY"
1,12086,1,wave_1,25.61,-80.50,Miami-Dade,FL,"Miami-Dade, FL"
2,48029,24,wave_2,29.45,-98.52,Bexar,TX,"Bexar, TX"
3,12011,25,wave_2,26.19,-80.48,Broward,FL,"Broward, FL"


In [43]:
# manually adjusting position on the map
df['position'] = 'top right'
df.position[df.RegionName.isin(['San Francesico'])]= 'middle center'
#df.position[df.RegionName.isin(['San Diego', 'Los Angeles'])]= 'middle left'
df.position[df.RegionName.isin(['San Diego', 'Los Angeles', 'Bronx'])]= 'top left'
df.position[df.RegionName.isin(['Santa Clara'])]= 'bottom left'
df.position[df.RegionName.isin(['Middlesex', 'Tarrant'])]= 'bottom right'
df.position[df.RegionName.isin(['Broward', 'San Mateo',  ])]='middle left'
df.position[df.RegionName.isin(['Kings'])]='bottom left'

In [44]:
# Initialize figure
fig = go.Figure()



# Add Traces

# --------------------------------------- wave 1 --------------------------------------- 
fig.add_trace(go.Scattergeo(
    locationmode = 'USA-states',
    lon = df[df.wave == 'wave_1'].lon,
    lat = df[df.wave == 'wave_1'].lat,
    text = df[df.wave == 'wave_1'].county,
    marker = dict(size = 0, line_color='rgb(40,40,40)',line_width=0.5),
    name= 'wave 1'))

fig.add_trace(go.Scattergeo(
    locationmode = 'USA-states',
    lon = df[df.wave == 'wave_1'].lon +.6,
    lat = df[df.wave == 'wave_1'].lat,
    #textposition = df[df.wave == 'wave_1'].position,
    textposition = 'bottom right',
    text = df[df.wave == 'wave_1'].RegionName,
    mode = 'text',
    ))

# --------------------------------------- wave 2 --------------------------------------- 


fig.add_trace(go.Scattergeo(
    locationmode = 'USA-states',
    lon = df[df.wave == 'wave_2'].lon,
    lat = df[df.wave == 'wave_2'].lat,
    text = df[df.wave == 'wave_2'].county,
    visible=False,
    marker = dict(size = 0, line_color='rgb(40,40,40)',line_width=0.5),
    name= 'wave 2'))

# adjusting manually postion for NYC counties
df.lon[df.RegionName.isin(['Broward', 'Bronx', 'Kings'])] = df.lon[df.RegionName.isin(['Broward', 'Bronx', 'Kings'])]  -1
df.lat[df.RegionName== 'Richmond'] = df.lat[df.RegionName== 'Richmond'] -1.5
df.lat[df.RegionName== 'Queens'] = df.lat[df.RegionName== 'Queens'] +0.2

fig.add_trace(go.Scattergeo(
    locationmode = 'USA-states',
    lon = df[df.wave == 'wave_2'].lon -.6,
    lat = df[df.wave == 'wave_2'].lat,
    textposition = df[df.wave == 'wave_2'].position,
    #textposition = 'middle left',
    text = df[df.wave == 'wave_2'].RegionName,
    visible=False,
    mode = 'text',
    ))

# --------------------------------------- wave 4 --------------------------------------- 


fig.add_trace(go.Scattergeo(
    
    locationmode = 'USA-states',
    lon = df[df.wave == 'wave_4'].lon,
    lat = df[df.wave == 'wave_4'].lat,
    text = df[df.wave == 'wave_4'].county,
    visible=False,
    marker = dict(size = 0, line_color='rgb(40,40,40)',line_width=0.5),
    name= 'wave 4'))

fig.add_trace(go.Scattergeo(
    locationmode = 'USA-states',
    lon = df[df.wave == 'wave_4'].lon +.6,
    lat = df[df.wave == 'wave_4'].lat,
    textposition = df[df.wave == 'wave_4'].position,
    text = df[df.wave == 'wave_4'].RegionName,
    visible=False,
    mode = 'text',
    ))

# --------------------------------------- wave 6 --------------------------------------- 


fig.add_trace(go.Scattergeo(
   
    locationmode = 'USA-states',
    lon = df[df.wave == 'wave_6'].lon,
    lat = df[df.wave == 'wave_6'].lat,
    text = df[df.wave == 'wave_6'].county,
    visible=False,
    marker = dict(size = 0, line_color='rgb(40,40,40)',line_width=0.5),
    name= 'wave 6'))

fig.add_trace(go.Scattergeo(
    locationmode = 'USA-states',
    lon = df[df.wave == 'wave_6'].lon +.6,
    lat = df[df.wave == 'wave_6'].lat,
    textposition = df[df.wave == 'wave_6'].position,
    text = df[df.wave == 'wave_6'].RegionName,
    visible=False,
    mode = 'text',
    ))


fig.update_geos(scope="usa")

# Add Buttons

fig.update_layout(showlegend=False,
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            active=0,
            x=0.8,
            y=1.2,
            buttons=list([
                dict(label="Wave 1",
                     method="update",
                     args=[{"visible": [True, True, False, False, False, False, False, False]},
                           {"title": "Wave 1 - Mar/2016"}]),
                dict(label="Wave 2",
                     method="update",
                     args=[{"visible": [True, True, True, True,False, False, False, False]},
                           {"title": "Wave 2 - Aug/2016"}]),
                dict(label="Wave 4",
                     method="update",
                     args=[{"visible": [True, True, True,True, True, True, False, False]},
                           {"title": "Wave 4 - Aug/2017<br>Wire transfers"}]),
                dict(label="Wave 6",
                     method="update",
                     args=[{"visible": [True, True, True, True, True, True, True, True]},
                           {"title": "Wave 6 - Fev/2018<br>US$300,000 and Cryptocurrencies"}]),
            ]),
        )
    ])

# Set title
fig.update_layout(
    title_text="GTO Waves",
    xaxis_domain=[0.05, 1.0],
)

fig.show()

In [14]:
fig.write_html("../images/jm_GTO_map_waves.html")

In [16]:
# workaround to find image size
fig.write_image("../images/fig1.png")

from PIL import Image
im=Image.open("../images/fig1.png")
im.size # (width,height) tuple

(700, 500)

### Organize zillow data to plot

In [79]:
# load price data and format Date as YYYY-MM
df = (pd.read_csv('../output/tbl_reg.csv',
                  dtype={'FIPS': object,'price_mid':'Int64',  'price_TOP':'Int64'},
                  usecols=['FIPS', 'Date', 'RegionName', 'State',
                           'price_mid', 'price_TOP',
                           'logprice_mid', 'logprice_TOP',
                           'price_pct_mid', 'price_pct_TOP',
                           ]))

df.Date = df.Date.str.slice(stop=7) # YYYY-MM date 
df.RegionName = df.RegionName.str.slice(stop=-7) # drop word "County"
df

,FIPS,Date,RegionName,State,price_mid,logprice_mid,price_pct_mid,price_TOP,logprice_TOP,price_pct_TOP
0,01001,2014-01,Autauga,AL,140007,11.85,0.00,217585,12.29,0.00
1,01001,2014-02,Autauga,AL,140229,11.85,0.00,217863,12.29,0.00
2,01001,2014-03,Autauga,AL,140479,11.85,0.00,218157,12.29,0.00
3,01001,2014-04,Autauga,AL,140779,11.85,0.00,218551,12.29,0.00
4,01001,2014-05,Autauga,AL,141139,11.86,0.00,219047,12.30,0.00
...,...,...,...,...,...,...,...,...,...,...
199570,56045,2019-11,Weston,WY,183719,12.12,0.00,273364,12.52,0.00
199571,56045,2019-12,Weston,WY,183927,12.12,0.00,273331,12.52,-0.00
199572,56045,2020-01,Weston,WY,184089,12.12,0.00,273177,12.52,-0.00
199573,56045,2020-02,Weston,WY,184259,12.12,0.00,273281,12.52,0.00


In [71]:
# Table in long form to plot
df_thres = df.drop(['logprice_mid', 'logprice_TOP','price_pct_mid', 'price_pct_TOP'],1).copy()
df_thres = pd.merge(df_thres , GTO.drop('date enforcement',1) , on = ['FIPS', 'Date'])
df_thres

,FIPS,Date,RegionName,State,price_mid,price_TOP,wave,threshold
0,06037,2017-08,Los Angeles,CA,590637,1069290,wave_4,2000000
1,06037,2018-11,Los Angeles,CA,635840,1148162,wave_6,300000
2,06073,2017-08,San Diego,CA,564548,889612,wave_4,2000000
3,06073,2018-11,San Diego,CA,600862,942417,wave_6,300000
4,06075,2017-08,San Francisco,CA,1271961,1969357,wave_4,2000000
5,06075,2018-11,San Francisco,CA,1413393,2129459,wave_6,300000
6,06081,2017-08,San Mateo,CA,1219278,2015865,wave_4,2000000
7,06081,2018-11,San Mateo,CA,1418645,2325701,wave_6,300000
8,06085,2017-08,Santa Clara,CA,1093447,1893576,wave_4,2000000
9,06085,2018-11,Santa Clara,CA,1298463,2211754,wave_6,300000


In [72]:
# keep track of order by threshold (to have a nice plot for 4th wave)
ordering_region = (df_thres[df_thres.wave=='wave_4'].
                    sort_values(['threshold','price_mid', 'price_TOP']).
                    loc[:,'RegionName'])
ordering_region

32            Bexar
10          Broward
14       Palm Beach
12       Miami-Dade
22            Bronx
28           Queens
30         Richmond
24            Kings
2         San Diego
0       Los Angeles
8       Santa Clara
6         San Mateo
4     San Francisco
16         Honolulu
26         New York
Name: RegionName, dtype: object

In [73]:
df_thres  = (df_thres.melt(id_vars=['FIPS', 'RegionName', 'State','Date', 'wave'], 
                           value_vars= ['price_mid', 'price_TOP', 'threshold'],
                           var_name='tier', value_name = 'value'))

In [74]:
# ordering categories: both plots will use this ordered
ordering = ["price_mid","price_TOP","threshold"]  
df_thres['tier'] = pd.Categorical(df_thres['tier'], categories=ordering) 
df_thres = df_thres.sort_values(['tier','value'])

In [75]:
# on a df copy, apply the order by thresholds (saved before) for the 4th wave, for a nice plot
df_4 = df_thres .copy()
df_4['RegionName'] = pd.Categorical(df_thres['RegionName'], categories=ordering_region) 
df_4 = df_4.sort_values(['RegionName'])

In [76]:
col_new[1]

'rgb(0.424750656167979, 0.8519160104986876, 0.8120472440944881)'

In [162]:
%run functions/nice_plot_pandas_getdata.ipynb

red = economist[-2]

fig = px.bar(df_4[(df_4.tier!='threshold') & (df_4.wave=='wave_4')], 
             y='value', 
             x='RegionName', #text='value', 
             color='tier', barmode='group',
             template="plotly_white",
             color_discrete_sequence=[ "lightgray", "#0EDBE9", red],   
             title="Median prices x thresholds<br>4th wave -  Aug/2017",
             #facet_row="wave", facet_col="State",
             labels= {'value':'US$', 'RegionName':' ', 'tier':' '}
            )

# horizontal lines
fig.add_shape(type="line",y0= 500000, y1= 500000, 
                          x0= -0.5, x1= 0.5,
                          line=dict(color=economist[-2], width=2))

fig.add_shape(type="line",y0= 1000000, y1= 1000000, 
                          x0= 0.5, x1= 3.5,
                          line=dict(color=red, width=2))

fig.add_shape(type="line",y0= 2000000, y1= 2000000, 
                          x0= 3.5, x1= 8.5,
                          line=dict(color=red, width=2))

fig.add_shape(type="line",y0= 3000000, y1= 3000000, 
                          x0= 8.5, x1= 14.5,
                          line=dict(color=red, width=2))

# vertical lines
fig.add_shape(type="line",y0= 0, y1= 1200000, 
                          x0= 0.5, x1= 0.5,
                          line=dict(color="gray", width=2, dash='dash'))

fig.add_shape(type="line",y0= 0, y1= 2200000, 
                          x0= 3.5, x1= 3.5,
                          line=dict(color="gray", width=2, dash='dash'))

fig.add_shape(type="line",y0= 0, y1= 3200000, 
                          x0= 8.5, x1= 8.5,
                          line=dict(color="gray", width=2, dash='dash'))

# add annotations
fig.add_trace(go.Scatter(
    x=['Bexar', 'Broward', 'San Diego', 'Honolulu'],
    y=[550000, 1050000, 2050000, 3050000],
    #x0=0, x1=1, y0=500000, y1=500000,
    
    mode="text",
    name="threshold",
    text=["500k", "1MM", "2MM", "3MM"],
    
    textposition="top center",
    textfont=dict(
        #family="sans serif",
        size=10,
        color="red"), 
    showlegend=False
))

# Source
annotations=[(dict(xref='paper', yref='paper', x=0, y=-0.16,
                                  xanchor='left', yanchor='top',
                                  text='Source: FinCEN / US Treasury',
                                  font=dict(family='Arial',size=12, color='rgb(150,150,150)'),
                                  showarrow=False))]


# longer y-axis
fig['layout']['yaxis1'].update(title='', range=[0, 3500000],autorange=False)
fig.update_layout(yaxis_title = 'US$', annotations=annotations)

fig.show()

In [163]:
#save plot 
fig.write_html("../images/jm_GTO_4th_wave.html")


In [49]:
# workaround to find image size
fig.write_image("../images/fig1.png")

from PIL import Image
im=Image.open("../images/fig1.png")
im.size # (width,height) tuple

(700, 500)

In [164]:
fig = px.bar(df_thres[(df_thres.tier!='threshold') & (df_thres.wave=='wave_6')], 
             y='value', 
             x='RegionName', #text='value', 
             color='tier', barmode='group',
             template="plotly_white",
             color_discrete_sequence=[ "lightgray","#0EDBE9", red],
             title="Median prices x thresholds<br>6th wave -  Aug/2018",
             labels= {'value':'US$', 'RegionName':' ', 'tier':' '}
            )

fig.add_shape(type="line",y0= 300000, y1= 300000, 
                          x0= -0.5, x1= 21.5,
                          line=dict(color=red, width=2))

# add annotations
fig.add_trace(go.Scatter(
    x=['Bexar'],
    y=[350000],
    mode="text",
    #name="threshold",
    text=["300k"],
    textposition="top center",
    textfont=dict(
        #family="sans serif",
        size=12,
        color=red), 
    showlegend=False
))

# Source
annotations=[(dict(xref='paper', yref='paper', x=0, y=-0.20,
                                  xanchor='left', yanchor='top',
                                  text='Source: FinCEN / US Treasury',
                                  font=dict(family='Arial',size=12, color='rgb(150,150,150)'),
                                  showarrow=False))]


fig.update_shapes(dict(xref='x', yref='y'))
fig.update_layout(annotations=annotations)

#fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()

In [165]:
#save plot 
fig.write_html("../images/jm_GTO_6th_wave.html")

In [26]:
# workaround to find image size
fig.write_image("../images/fig1.png")

from PIL import Image
im=Image.open("../images/fig1.png")
im.size # (width,height) tuple

(700, 500)

## Time series for each treated county 
For each treated couty, I plot the TS (mid and top) and sales, when available, and the treatment dates.

I will start without control variables.

In [148]:
dfM =load_Zillow_YYYYMM_noCounty()
dfM

,FIPS,Date,RegionName,State,price_mid,logprice_mid,price_pct_mid,price_TOP,logprice_TOP,price_pct_TOP
0,01001,2014-01,Autauga,AL,140007,11.85,0.00,217585,12.29,0.00
1,01001,2014-02,Autauga,AL,140229,11.85,0.00,217863,12.29,0.00
2,01001,2014-03,Autauga,AL,140479,11.85,0.00,218157,12.29,0.00
3,01001,2014-04,Autauga,AL,140779,11.85,0.00,218551,12.29,0.00
4,01001,2014-05,Autauga,AL,141139,11.86,0.00,219047,12.30,0.00
...,...,...,...,...,...,...,...,...,...,...
199570,56045,2019-11,Weston,WY,183719,12.12,0.00,273364,12.52,0.00
199571,56045,2019-12,Weston,WY,183927,12.12,0.00,273331,12.52,-0.00
199572,56045,2020-01,Weston,WY,184089,12.12,0.00,273177,12.52,-0.00
199573,56045,2020-02,Weston,WY,184259,12.12,0.00,273281,12.52,0.00


In [150]:
# prepare data to plot

def prep_df_plot(target_vars=['price_mid', 'price_TOP'], df1 = df, level = 'price'):
    
    # drop vars, make long table and append tables for prices and % changes
    
    var_list = ['FIPS', 'Date', 'RegionName', 'State'] + target_vars
    dfM = df.copy()
    #print(dfM.columns)
    dfM = dfM[var_list]
    dfM.rename(columns={target_vars[0]:'mid',target_vars[1]:'top'}, inplace= True)
    #print(dfM.columns)
    dfM = dfM.melt(id_vars=['FIPS', 'Date', 'RegionName', 'State'], 
                               value_vars=['mid','top'],
                               var_name = 'tier',
                               value_name = 'value')
    dfM['level']=level

    return(dfM)
    
df_price = prep_df_plot(target_vars=['price_mid', 'price_TOP'])
df_growth =  prep_df_plot(target_vars=['price_pct_mid', 'price_pct_TOP'], level='%change')

In [151]:
dfM =  df_price.append(df_growth)
dfM

,FIPS,Date,RegionName,State,tier,value,level
0,01001,2014-01,Autauga,AL,mid,140007,price
1,01001,2014-02,Autauga,AL,mid,140229,price
2,01001,2014-03,Autauga,AL,mid,140479,price
3,01001,2014-04,Autauga,AL,mid,140779,price
4,01001,2014-05,Autauga,AL,mid,141139,price
...,...,...,...,...,...,...,...
399145,56045,2019-11,Weston,WY,top,0.00,%change
399146,56045,2019-12,Weston,WY,top,-0.00,%change
399147,56045,2020-01,Weston,WY,top,-0.00,%change
399148,56045,2020-02,Weston,WY,top,0.00,%change


In [233]:
def plot_series (title= "House prices", df1=dfM, level='price', state=['NY']):

    # if working only with treated counties - can work with many other selections
    dfM = df1.loc[(df1.FIPS.isin(GTO.FIPS.unique())& (df1.State.isin(state)))]
    dfM = dfM[dfM.level==level]
    
    # number of lines and row on faceted plot - 
    # now I am setting wrap to number of states 
    #so I dont need to loop over rows, but I kept the code.
    n_county = len(dfM.RegionName.unique())
    # col_wrap = n_county # uncomment if you dont to wrap by same number of counties
    col_wrap = n_county
    n_row = n_county // col_wrap # integer part
    
    # title only plot prices
    title = ((title + ': '+ ', '.join(state)) if level=='price' else '')
    
    fig = px.line(dfM,          
              x="Date", 
              y='value',
              color='tier',
              title=title,
              #facet_row="State",
              facet_col='RegionName', 
              facet_col_wrap=col_wrap,
              template="plotly_white",
              #color_discrete_sequence=['C3CBF9''#053061'],
              color_discrete_sequence=['#00CBF9','#0E4D92'],
              
              labels= {'State':'','Date':'', 'RegionName':'', 'value':'',
                      },)
    
    # min and max of y - to make vertical lines
    min_y = min(dfM.value)*.9
    max_y = max(dfM.value)*1.1
    min_x = min(dfM.Date)
    max_x = max(dfM.Date)
    
    # vertical lines on every subplot:
    
    def make_line(date, row, col, color='gray', dash='dash'):  
        fig.add_shape(type='line',y0= min_y, y1= max_y, 
                      x0= date, x1= date,
                      row=row, col=col,
                      line=dict(color=color, width=1, dash=dash))
    
    
    for n in list(range(n_row+1)):
        for m in list(range(col_wrap)):
            if ((n_county%col_wrap != 0) & (n==n_row & m==col_wrap)):
                pass
            else:
                make_line('2018-8', row=n, col=m, color='red') # 6th wave    

                        
                # add horizontal line at y = 0 on percentage change prices
                if level=='%change':
                    fig.add_shape(type='line',y0= 0, y1= 0, 
                      x0= min_x, x1= max_x,
                      row=n, col=m,
                      line=dict(color="darkgray", width=.5))
                
    # vertical line on 2 first treatments on NY and Florida + first treatment on the rest
    if 'NY' in state:
        loc = list(dfM.RegionName.unique()).index('New York') + 1
        make_line('2016-3', row=0, col=loc)    # first wave
        for county in ['Bronx', 'Kings', 'New York', 'Queens','Richmond']:
            loc = list(dfM.RegionName.unique()).index(county) + 1
            make_line('2016-8', row=0, col=loc) # 2nd 
            make_line('2017-8', row=0, col=loc, dash='dot', color='red') # 4th
        
    elif 'FL'in state:
        loc = list(dfM.RegionName.unique()).index('Miami-Dade') + 1
        make_line('2016-3', row=0, col=loc)     # first wave
        for county in ['Broward', 'Miami-Dade', 'Palm Beach']:
            loc = list(dfM.RegionName.unique()).index(county) + 1
            make_line('2016-8', row=0, col=loc) # 2nd 
            make_line('2017-8', row=0, col=loc, dash='dot', color='red') # 4th

            
    elif 'CA'in state:
        for county in ['Los Angeles', 'San Diego', 'San Francisco', 'San Mateo',
                       'Santa Clara']:
            loc = list(dfM.RegionName.unique()).index(county) + 1
            make_line('2016-8', row=0, col=loc) # 2nd 
            make_line('2017-8', row=0, col=loc, dash='dot', color='red') # 4th
      
    elif 'TX'in state:
        for county in ['Bexar']:
            loc = list(dfM.RegionName.unique()).index(county) + 1
            make_line('2016-8', row=0, col=loc) # 2nd 
            make_line('2017-8', row=0, col=loc, dash='dot', color='red') # 4th

    elif 'HI'in state:
        for county in ['Honolulu']:
            loc = list(dfM.RegionName.unique()).index(county) + 1
            make_line('2017-8', row=0, col=loc, dash='dot', color='red') # 4tH

    width = 172*col_wrap +150
    height=(295 if level=='price' else 265) # ternary operator 
   
    #adjust margins to fit reveal.js
    margin=dict(autoexpand=False,b=40, t=50) # margin larger to fit "source"
    
    # Add source
    if level=='%change':
        annotations=[(dict(xref='paper', yref='paper', x=0, y=-.3,
                                  xanchor='left', yanchor='top',
                                  text='Source: FinCEN / US Treasury',
                                  font=dict(family='Arial',size=12, color='rgb(150,150,150)'),
                                showarrow=False))]
        
        margin=dict(autoexpand=False,t=10, b=70) # margin larger to fit "source"
        fig.update_layout(annotations=annotations)
    
        
    fig.update_layout(autosize=False, width=width, height=height,margin= margin)
    fig.update_xaxes(tickangle=-90, nticks=2,)
    
    # Show only values on facet labels
    fig.for_each_annotation(lambda a: a.update(text=a.text.replace("=", "")))
    
    return fig

In [234]:
fig1 = plot_series(level='price', state=['NY'])
fig2 = plot_series(title='% price change', level='%change', state=['NY'])

In [235]:
def plot_TS():
    for states in [['NY'],['FL','MA'], ['CA'],['NV', 'HI','TX'],['WA', 'IL']]:
        fig1 = plot_series(level='price', state=states)
        fig2 = plot_series(title='% price change', level='%change', state=states)
        fig1.write_html("../images/jm_GTO_TS1_" + states[0] + ".html")
        fig2.write_html("../images/jm_GTO_TS2_" + states[0] + ".html")
        
        # workaround to find image size
        fig1.write_image("../images/fig1.png")
        im=Image.open("../images/fig1.png")
        print("level", states[0],  im.size) # (width,height) tuple
        
        fig2.write_image("../images/fig1.png")
        im=Image.open("../images/fig1.png")
        print("% change", states[0],  im.size) # (width,height) tuple

The plot show the time series of house prices and price changes for each treated county. I mark specific dates with vertical lines:
* dashed gray: first time treated (also marked 2nd time for Miami and NYC)
* dotted red: inclusion of wire transfers (4th wave - Aug/2017)
* dashed red: threshold = US$300,000 and incl. criptocurrencies(6th wave - Aug/2017)
    

In [236]:
plot_TS()

level NY (1010, 295)
% change NY (1010, 265)
level FL (1010, 295)
% change FL (1010, 265)
level CA (1010, 295)
% change CA (1010, 265)
level NV (1010, 295)
% change NV (1010, 265)
level WA (494, 295)
% change WA (494, 265)
